# Feedforward NN

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [2]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
print(f'The following device is being used: {DEVICE}')

The following device is being used: cpu


## Configurations

In [4]:
# params
epochs = 10
batch_size = 32

input_size = 28*28
hidden_1 = 100
hidden_2 = 50
output_size = 10
lr = 0.01

## Getting the data

In [5]:
# dataset
train_mnist = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_mnist = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

In [6]:
# dataloader
train_dataloader = DataLoader(dataset=train_mnist, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(dataset=train_mnist, batch_size=batch_size, shuffle=True)

## MNIST Fully Connected Neural Network

In [7]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.fc3 = nn.Linear(hidden_2, output_size)
    
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return x

In [8]:
model = Model()
model.to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [9]:
for epoch in range(epochs):
    print(f'Epoch Nr: {epoch}')
    for images, labels in train_dataloader:
        images = images.view(-1, input_size)
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)
        
        preds = model(images)
        loss = criterion(preds, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

Epoch Nr: 0
Epoch Nr: 1
Epoch Nr: 2
Epoch Nr: 3
Epoch Nr: 4
Epoch Nr: 5
Epoch Nr: 6
Epoch Nr: 7
Epoch Nr: 8
Epoch Nr: 9


In [11]:
# calculate accuracy
with torch.no_grad():
    correct = 0
    count = 0
    for images, labels in test_dataloader:
        images = images.view(-1, input_size)
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)
        prediction = model(images)
        prediction = torch.argmax(prediction, dim=1)
        count += labels.shape[0]
        correct += torch.eq(prediction, labels).sum().item()
        
    accuracy = correct / count
    print(accuracy)

0.9690833333333333
